In [1]:
import datetime
import logging
import os
import numpy as np
import torch
from importlib import reload

from models import dapm
from scripts.data_loader import *
from scripts.train_dapm import train
from utils.metrics import normalize_mat
from params import Param
from utils.logging_utils import *

import warnings
warnings.filterwarnings('ignore')


In [2]:
import torch
import torch.nn as nn

from models.fc import FC
from models.auto_encoder import AutoEncoder
from models.mask_net import MaskNet
from models.feature_attention import FeatureAttention


class DeepAir(nn.Module):

    def __init__(self, in_dim, ae_en_h_dims, ae_de_h_dims, fc_in_dim, fc_h_dims, fc_out_dim, **kwargs):

        super(DeepAir, self).__init__()

        self.kwargs = kwargs
        self.device = kwargs.get('device', 'cpu')

        ################
        # sparse layer #
        ################
        
        mask_indices = [[range(in_dim)], [range(in_dim)]]
        self.mask_layer = MaskNet(in_dim, in_dim, mask_indices=mask_indices, device=self.device)
        self.mask_thre = kwargs.get('mask_thre', 0.0001)
        
        ######################
        # auto_encoder layer #
        ######################

        self.ae = AutoEncoder(in_dim=in_dim,
                              en_h_dims=ae_en_h_dims,
                              de_h_dims=ae_de_h_dims)

        if kwargs.get('ae_pretrain_weight') is not None:
            self.ae.load_state_dict(kwargs['ae_pretrain_weight'])
        else:
            raise ValueError('AutoEncoder not pretrained.')

        for p in self.ae.parameters():
            p.requires_grad = True

        #########################
        # fully-connected layer #
        #########################

        self.fc = FC(in_dim=fc_in_dim,  # assert in_dim == n_conv_lstm * conv_lstm_h_dim
                     h_dims=fc_h_dims,
                     out_dim=fc_out_dim,
                     p_dropout=kwargs.get('fc_p_dropout', 0.1))

    def forward(self, input_data):  # input_data: (b, t, c, h, w)

        x = input_data.permute(0, 1, 3, 4, 2)  # => (b, t, h, w, c)
        x = x[:, -1, ...]

        ################
        # sparse layer #
        ################

        masked_x = self.mask_layer(x)
        for p in self.mask_layer.parameters():
            for i in range(p.shape[0]):
                if -self.mask_thre <= p[i, i] <= self.mask_thre:
                    masked_x[..., i] = 0.0

        ######################
        # auto-encoder layer #
        ######################

        en_x, de_x = self.ae(masked_x)        

        #########################
        # fully-connected layer #
        #########################

        fc_out = self.fc(en_x)
        fc_out = fc_out.permute(0, 3, 1, 2)  # => (b, c, h, w)

        return fc_out, masked_x, en_x, de_x


In [17]:
import logging
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as dat
from tensorboardX import SummaryWriter

from utils.early_stopping import EarlyStopping
from utils.metrics import compute_error
from models.spatial_loss_func import SpatialLossFunc


def train(dapm, data_obj, args, **kwargs):
    
    """ construct index-based data loader """
    idx = np.array([i for i in range(args.seq_len, data_obj.train_y.shape[0])])
    idx_dat = dat.TensorDataset(torch.tensor(idx, dtype=torch.int32))
    train_idx_data_loader = dat.DataLoader(dataset=idx_dat, batch_size=args.batch_size, shuffle=True)
    
    idx = np.array([i for i in range(args.seq_len, data_obj.test_y.shape[0])])
    idx_dat = dat.TensorDataset(torch.tensor(idx, dtype=torch.int32))
    test_idx_data_loader = dat.DataLoader(dataset=idx_dat, batch_size=1, shuffle=False)

    """ set writer, loss function, and optimizer """
    writer = SummaryWriter(kwargs['run_file'])
    loss_func = nn.MSELoss()
    spatial_loss_func = SpatialLossFunc(sp_neighbor=args.sp_neighbor) 
    optimizer = optim.Adam(dapm.parameters(), lr=args.lr, weight_decay=1e-8)
    early_stopping = EarlyStopping(patience=5, verbose=True)

    def get_current_learning_rate(optimizer):
        for param_group in optimizer.param_groups:
            return param_group['lr']
        
    def adjust_learning_rate(optimizer, epoch):
        """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
        if epoch < 30:
            lr = args.lr * (0.1 ** (epoch // 10))
        else:
            lr = get_current_learning_rate(optimizer)
        print(lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

            
    for epoch in range(args.epochs):

        dapm.train()
        total_losses, train_losses, val_losses = [], [], []

        adjust_learning_rate(optimizer, epoch)

        for _, idx in enumerate(train_idx_data_loader):
            batch_idx = idx[0]

            ############################
            # construct sequence input #
            ############################

            def construct_sequence_x(idx_list, dynamic_x, static_x):
                d_x = [dynamic_x[i - args.seq_len: i + 1, ...] for i in idx_list]
                d_x = np.stack(d_x, axis=0)
                s_x = np.expand_dims(static_x, axis=0)
                s_x = np.repeat(s_x, args.seq_len + 1, axis=1)  # (t, c, h, w)
                s_x = np.repeat(s_x, len(idx_list), axis=0)  # (b, t, c, h, w)
                x = np.concatenate([d_x, s_x], axis=2)
                return torch.tensor(x, dtype=torch.float).to(kwargs['device'])

            def construct_y(idx_list, output_y):
                y = [output_y[i] for i in idx_list]
                y = np.stack(y, axis=0)
                return torch.tensor(y, dtype=torch.float).to(kwargs['device'])


            batch_x = construct_sequence_x(batch_idx, data_obj.dynamic_x, data_obj.static_x)  # x = (b, t, c, h, w)
            batch_y = construct_y(batch_idx, data_obj.train_y)  # y = (b, c, h, w)
            batch_val_y = construct_y(batch_idx, data_obj.val_y)

            ###################
            # train the model #
            ###################

            out, masked_x, _, de_x = dapm(batch_x)
            train_loss = loss_func(batch_y[~torch.isnan(batch_y)], out[~torch.isnan(batch_y)])
            train_losses.append(train_loss.item())

            # add loss according to the model type
            total_loss = train_loss
            if 'sp' in args.model_type:
                mask_layer_params = torch.cat([x.view(-1) for x in dapm.mask_layer.parameters()])
                l1_regularization = torch.norm(mask_layer_params, 1)
                total_loss += l1_regularization * args.alpha

            if 'ae' in args.model_type:
                ae_loss = loss_func(masked_x, de_x)
                total_loss += ae_loss * args.gamma
                
            if 'sc' in args.model_type:
                sp_loss = spatial_loss_func(out)
                total_loss += sp_loss * args.beta

            total_losses.append(total_loss.item())

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
#             scheduler.step()

            ######################
            # validate the model #
            ######################

            val_loss = loss_func(batch_val_y[~torch.isnan(batch_val_y)], out[~torch.isnan(batch_val_y)])
            val_losses.append(val_loss.item())
        

        avg_total_loss = np.average(total_losses)
        avg_train_loss = np.average(train_losses)
        avg_val_loss = np.average(val_losses)

        # write for tensorboard visualization
        writer.add_scalar('data/train_loss', avg_total_loss, epoch)
        writer.add_scalar('data/val_loss', avg_val_loss, epoch)

        logging.info(f'Epoch [{epoch}/{args.epochs}] total_loss = {avg_total_loss:.4f}, train_loss = {avg_train_loss:.4f}, valid_loss = {avg_val_loss:.4f}.')

        ##################
        # early_stopping #
        ##################

        early_stopping(avg_val_loss, dapm, kwargs['model_file'])

        #########################
        # evaluate testing data #
        #########################
        
        if early_stopping.counter < 2 and epoch % 2 == 0:
            
            dapm.eval()
            predictions = []

            with torch.no_grad():
                for i, data in enumerate(test_idx_data_loader):
                    batch_idx = data[0]
                    batch_x = construct_sequence_x(batch_idx, data_obj.dynamic_x, data_obj.static_x)  # x = (b, t, c, h, w)
                    out, _, _, _ = dapm(batch_x)
                    predictions.append(out.cpu().data.numpy())

            prediction = np.concatenate(predictions)
            rmse, mape, r2 = compute_error(data_obj.test_y[args.seq_len:, ...], prediction)
            writer.add_scalar('data/test_rmse', rmse, epoch)
            logging.info(f'Testing: RMSE = {rmse:.4f}, MAPE = {mape:.4f}, R2 = {r2:.4f}.')

        if early_stopping.early_stop:
            logging.info(kwargs['model_name'] + f' val_loss = {early_stopping.val_loss_min:.4f}.')
            logging.info('Early stopping')
            break


In [18]:

def dapm_main(param, **kwargs):
    
    """ define model name """ 
    model_name = param.generate_model_name()
    ae_model_name = param.generate_ae_model_name()
    print(model_name)
    print(ae_model_name)

    kwargs['model_name'] = model_name
    kwargs['model_file'] = os.path.join(kwargs['model_dir'], model_name + '.pkl')
    kwargs['log_file'] = os.path.join(kwargs['log_dir'], model_name + '.log')
    kwargs['run_file'] = os.path.join(kwargs['run_dir'], model_name + '_run_{}'.format(datetime.datetime.now().strftime('%d%H%m')))
    kwargs['ae_model_file'] = os.path.join('./data/ae_models_2/models/', ae_model_name + '.pkl')

    """ load data """
    data_dir = f'/home/yijun/notebooks/training_data/'
    data_obj = load_data(data_dir, param)
    train_loc, val_loc, test_loc = load_locations(kwargs['train_val_test'], param)
    
    data_obj.train_loc = train_loc
    data_obj.train_y = data_obj.gen_train_val_test_label(data_obj.label_mat, data_obj.train_loc)
    data_obj.val_loc = val_loc
    data_obj.val_y = data_obj.gen_train_val_test_label(data_obj.label_mat, data_obj.val_loc)
    data_obj.test_loc = test_loc
    data_obj.test_y = data_obj.gen_train_val_test_label(data_obj.label_mat, data_obj.test_loc)
    
    """ logging starts """
    start_logging(kwargs['log_file'], model_name)
    data_logging(data_obj)

    """ load ae model """
    ae = torch.load(kwargs['ae_model_file'])
    
    """ define DeepAP model
    in_dim, ae_en_h_dims, ae_de_h_dims
    conv_lstm_in_size, conv_lstm_in_dim, conv_lstm_h_dim, conv_lstm_kernel_sizes, conv_lstm_n_layers
    fc_in_dim, fc_h_dims, fc_out_dim  """
    m = DeepAir(in_dim=data_obj.n_features,
                ae_en_h_dims=param.ae_en_h_dims,
                ae_de_h_dims=param.ae_de_h_dims,
                               
                fc_in_dim=param.ae_en_h_dims[-1],
                fc_h_dims=param.fc_h_dims,  # fc_h_dims
                fc_out_dim=1,
                                    
                ae_pretrain_weight=ae.state_dict(),
                mask_thre=param.mask_thre,
                fc_p_dropout=0.1,
                device=kwargs['device'])
   
    m = m.to(kwargs['device'])
    train(m, data_obj, param, **kwargs)
    
    """ logging ends """
    end_logging(model_name)
    

In [19]:
"""
    define directory
"""

base_dir = f'data/deep_air_1234_tp1/'
train_val_test_file = f'/home/yijun/notebooks/training_data/train_val_test_los_angeles_500m_fine_tune_1234.json'
device = torch.device("cuda:2" if torch.cuda.is_available() else 'cpu')  # the gpu device

""" load train, val, test locations """
f = open(train_val_test_file, 'r')
train_val_test = json.loads(f.read())

kwargs = {
    'model_dir': os.path.join(base_dir, 'models/'),
    'log_dir': os.path.join(base_dir, 'logs/'),
    'run_dir': os.path.join(base_dir, 'runs/'),
    'train_val_test': train_val_test,
    'device': device
}


In [20]:
for m in range(7, 8):
    param = Param([m], 2018, alpha=0.5, beta=0.1, gamma=5, eta=0, fc_h_dims=[8], sp_neighbor=2, mask_thre=0.0001, lr=0.01, model_type=['sp', 'ae', 'sc'])
    dapm_main(param, **kwargs)           


dapm___sp_ae_sc___los_angeles_500m_2018___#07#___6_00001_2___05_01_5_0___8_13
ae___los_angeles_500m_2018___#07#___16
0.01
0.01
0.01
0.01
0.01
0.01
0.01
0.01
0.01
0.01
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.00010000000000000002
0.00010000000000000002
0.00010000000000000002
0.00010000000000000002
0.00010000000000000002
0.00010000000000000002
0.00010000000000000002
0.00010000000000000002
